In [1]:
import numpy as np 
import pandas as pd 
import torch
from data_processing import generate_vocab, process_data, create_dataloaders
from model import get_pretrained_emb, EncoderRNN, DecoderRNN, DecoderAttnRNN, EncoderDecoder, EncoderDecoderAttn
from train_eval import train_and_eval, count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
import pickle as pkl 
from datetime import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# params required for generating data loaders 

SRC_LANG = 'vi'
TARG_LANG = 'en'

SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 
TARG_VOCAB_SIZE = 30000 

BATCH_SIZE = 64 

In [3]:
# takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [4]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, filter_long=False)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE, filter_long=False) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000, filter_long=False)

In [5]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [6]:
# model architecture params 
NETWORK_TYPE = 'rnn'
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 
DEC_DROPOUT = 0.2  
ATTENTION_TYPE = 'additive'

# training params  
NUM_EPOCHS = 15 
LR = 0.0001 
OPTIMIZER = 'Adam'
LAZY_TRAIN = False

# name the model and experiment 
if NETWORK_TYPE == 'rnn': 
    EXPERIMENT_NAME = '{}-rnn-{}-attn'.format(SRC_LANG, ATTENTION_TYPE)
elif NETWORK_TYPE == 'cnn': 
    EXPERIMENT_NAME = '{}-cnn'.format(SRC_LANG)
MODEL_NAME = '{}-{}'.format(EXPERIMENT_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [7]:
# store as dict to save to results later 
params = {'experiment_name': EXPERIMENT_NAME,'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 
          'rnn_cell_type': RNN_CELL_TYPE, 'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 
          'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 'src_vocab_size': SRC_VOCAB_SIZE, 
          'targ_vocab_size': TARG_VOCAB_SIZE, 'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 
          'dec_hidden_dim': DEC_HIDDEN_DIM, 'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 
          'clip_grad_max_norm': CLIP_GRAD_MAX_NORM, 'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 
          'attention_type': ATTENTION_TYPE, 'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 
          'learning_rate': LR, 'optimizer': OPTIMIZER, 'lazy_train': LAZY_TRAIN} 

In [8]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

if ATTENTION_TYPE == 'without': 
    # without attention 
    decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
                         targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], 
                                                                vocab[TARG_LANG]['token2id']))
    model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']).to(device)
    
else: 
    # with attention 
    decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                             num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, 
                             src_max_sentence_len=SRC_MAX_SENTENCE_LEN, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                             dec_dropout=DEC_DROPOUT, attention_type=ATTENTION_TYPE,
                             pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], 
                                                                    vocab[TARG_LANG]['token2id']))
    model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']).to(device) 

In [ ]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=100, save_checkpoint=True, save_to_log=True, 
    lazy_eval=True, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 0.00, Val Loss: 10.29, Train BLEU: 0.00, Val BLEU: 0.00, Minutes Elapsed: 0.13
Sampling from val predictions...
Source: chúng_ta những nạn_nhân cần đến tất_cả mọi người . <EOS>
Reference: we victims need everyone . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> caption caption caption caption elitist elitist elitist freely freely
Attention Weights: tensor([[0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000,

Epoch: 0.10, Train Loss: 0.00, Val Loss: 6.21, Train BLEU: 0.00, Val BLEU: 1.30, Minutes Elapsed: 5.19
Sampling from val predictions...
Source: các gia_đình hàng_xóm nghe kể về ý_tưởng này . <EOS>
Reference: and my neighboring homes heard about this idea .
Model: <SOS> and &apos;s the the . . . . .
Attention Weights: tensor([[8.3157e-06, 4.1053e-03, 8.9195e-02, 2.5529e-01, 2.5606e-01, 2.0147e-01,
         1.3048e-01, 5.1371e-02, 9.3558e-03, 2.6709e-03],
        [8.7240e-05, 1.3469e-02, 1.0965e-01, 2.1270e-01, 2.0989e-01, 1.7924e-01,
         1.3910e-01, 8.2685e-02, 3.1731e-02, 2.1444e-02],
        [1.4440e-04, 5.4484e-03, 2.8936e-02, 5.1302e-02, 5.0970e-02, 4.5558e-02,
         4.0251e-02, 3.4335e-02, 3.8145e-02, 7.0491e-01],
        [1.2625e-05, 3.3452e-04, 1.8714e-03, 3.4943e-03, 3.4164e-03, 3.0291e-03,
         2.7818e-03, 2.7626e-03, 5.7113e-03, 9.7659e-01],
        [4.1132e-06, 1.0739e-04, 6.6137e-04, 1.2866e-03, 1.2309e-03, 1.0732e-03,
         9.9927e-04, 1.0725e-03, 3.0761e-03,

Epoch: 0.14, Train Loss: 0.00, Val Loss: 6.15, Train BLEU: 0.00, Val BLEU: 1.36, Minutes Elapsed: 7.69
Sampling from val predictions...
Source: bây_giờ tôi muốn giới_thiệu các bạn với những người em_trai
Reference: now i &apos;d like to introduce you to my
Model: <SOS> and , , , , , , , ,
Attention Weights: tensor([[1.7282e-06, 3.5568e-05, 1.3156e-03, 3.7219e-02, 1.7338e-01, 3.2576e-01,
         3.0054e-01, 1.5262e-01, 9.1044e-03, 2.7114e-05],
        [1.7126e-05, 3.0989e-04, 6.4648e-03, 7.2837e-02, 1.9150e-01, 2.8140e-01,
         2.6474e-01, 1.6437e-01, 1.8301e-02, 6.2360e-05],
        [1.4059e-04, 1.7037e-03, 1.8358e-02, 9.8321e-02, 1.8429e-01, 2.3224e-01,
         2.2533e-01, 1.7845e-01, 5.9534e-02, 1.6301e-03],
        [5.3611e-04, 3.6287e-03, 2.3545e-02, 9.4603e-02, 1.6717e-01, 2.0627e-01,
         2.0373e-01, 1.7875e-01, 1.0259e-01, 1.9182e-02],
        [7.2082e-04, 3.8620e-03, 2.1698e-02, 8.5845e-02, 1.5745e-01, 1.9778e-01,
         1.9591e-01, 1.7526e-01, 1.1587e-01, 4.5603e-0

Epoch: 0.19, Train Loss: 0.00, Val Loss: 6.14, Train BLEU: 0.00, Val BLEU: 1.44, Minutes Elapsed: 10.19
Sampling from val predictions...
Source: cô cũng đã có bản_sao của bức ảnh . <EOS>
Reference: she also had <UNK> . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> and &apos;s is the the . . . .
Attention Weights: tensor([[3.1292e-05, 3.6202e-03, 9.7324e-02, 3.3806e-01, 3.9314e-01, 1.3732e-01,
         2.6389e-02, 3.2184e-03, 5.8511e-04, 3.1986e-04],
        [1.8756e-04, 1.2437e-02, 1.4713e-01, 3.3343e-01, 3.6238e-01, 1.2223e-01,
         2.0536e-02, 1.5367e-03, 1.1547e-04, 1.8871e-05],
        [3.5653e-04, 2.0291e-02, 1.5693e-01, 2.9468e-01, 3.1801e-01, 1.5951e-01,
         4.4742e-02, 5.0739e-03, 3.6289e-04, 4.0808e-05],
        [1.1531e-03, 2.9656e-02, 1.3656e-01, 2.1912e-01, 2.4009e-01, 1.9036e-01,
         1.1862e-01, 4.6776e-02, 1.2559e-02, 5.1102e-03],
        [9.0547e-04, 1.5486e-02, 6.8923e-02, 1.1573e-01, 1.3175e-01, 1.2594e-01,
         1.1540e-01, 1.0272e-01, 1.0496e-01, 2.1819e-01],

Epoch: 0.24, Train Loss: 0.00, Val Loss: 6.09, Train BLEU: 0.00, Val BLEU: 1.22, Minutes Elapsed: 12.69
Sampling from val predictions...
Source: bức này được chụp vài tuần sau sự_kiện 11/9 ,
Reference: this one was taken just weeks after 9 /
Model: <SOS> so i , , , , , , ,
Attention Weights: tensor([[1.0457e-06, 1.5149e-05, 6.0338e-04, 2.2053e-02, 1.4643e-01, 4.2314e-01,
         3.2781e-01, 7.0959e-02, 8.1638e-03, 8.2759e-04],
        [4.2169e-06, 7.0708e-05, 2.3426e-03, 4.7131e-02, 1.8065e-01, 3.9584e-01,
         2.9627e-01, 7.0941e-02, 6.4913e-03, 2.6289e-04],
        [9.4693e-06, 1.7831e-04, 4.9891e-03, 6.2688e-02, 1.8065e-01, 3.1932e-01,
         2.8208e-01, 1.2204e-01, 2.5975e-02, 2.0665e-03],
        [1.2596e-04, 9.5308e-04, 8.7100e-03, 4.9105e-02, 1.1280e-01, 1.7860e-01,
         1.9239e-01, 1.6182e-01, 1.4628e-01, 1.4921e-01],
        [1.1593e-04, 5.2860e-04, 3.1579e-03, 1.5769e-02, 3.9052e-02, 6.7206e-02,
         8.0280e-02, 8.3599e-02, 1.4264e-01, 5.6765e-01],
        [7.1

...

Epoch: 0.34, Train Loss: 0.00, Val Loss: 6.04, Train BLEU: 0.00, Val BLEU: 2.23, Minutes Elapsed: 17.67
Sampling from val predictions...
Source: đây là những ý_tưởng cần_thiết mà một đất_nước đã bị
Reference: these are the ideals that a war-torn libya needs
Model: <SOS> it &apos;s a a a a a a a
Attention Weights: tensor([[8.1807e-04, 4.2498e-02, 5.2140e-01, 3.9098e-01, 4.3869e-02, 4.0526e-04,
         2.7698e-05, 5.3590e-06, 6.8273e-07, 2.7185e-07],
        [4.3248e-03, 9.0189e-02, 5.3184e-01, 3.2518e-01, 4.7631e-02, 7.6097e-04,
         6.6216e-05, 1.3734e-05, 2.0928e-06, 9.0415e-07],
        [2.2520e-03, 3.7600e-02, 2.4581e-01, 4.2283e-01, 2.7163e-01, 1.7383e-02,
         2.0320e-03, 4.1716e-04, 3.8149e-05, 1.2422e-05],
        [1.0343e-03, 4.4474e-03, 1.7510e-02, 6.3187e-02, 2.2266e-01, 2.5937e-01,
         2.0730e-01, 1.6194e-01, 4.2664e-02, 1.9886e-02],
        [2.0720e-04, 8.9477e-04, 4.0513e-03, 2.0703e-02, 1.0846e-01, 2.0928e-01,
         2.3817e-01, 2.4812e-01, 1.0692e-01, 6.3

Epoch: 0.38, Train Loss: 0.00, Val Loss: 5.99, Train BLEU: 0.00, Val BLEU: 2.75, Minutes Elapsed: 20.16
Sampling from val predictions...
Source: có một người phụ_nữ địa_phương tuyệt_vời đã hướng_dẫn chúng_tôi .
Reference: we had an amazing local woman who guided us
Model: <SOS> so &apos;s is the the the . . <EOS>
Attention Weights: tensor([[1.1766e-05, 1.6281e-03, 7.7843e-02, 5.0699e-01, 3.3741e-01, 7.4112e-02,
         1.7175e-03, 2.4495e-04, 2.8698e-05, 8.2875e-06],
        [1.6196e-04, 8.2038e-03, 1.2017e-01, 4.5895e-01, 3.3688e-01, 7.3145e-02,
         2.2353e-03, 2.3913e-04, 1.5733e-05, 2.6149e-06],
        [1.2515e-04, 7.2213e-03, 1.1189e-01, 3.6483e-01, 3.3615e-01, 1.6235e-01,
         1.4132e-02, 2.9679e-03, 2.9092e-04, 5.0445e-05],
        [4.7738e-05, 4.4748e-04, 4.7278e-03, 1.5168e-02, 1.8068e-02, 3.3904e-02,
         2.2382e-02, 4.8073e-02, 1.5776e-01, 6.9942e-01],
        [1.0438e-06, 4.8754e-06, 3.8656e-05, 1.2309e-04, 1.5416e-04, 4.9599e-04,
         6.2262e-04, 3.7698e-

Epoch: 0.43, Train Loss: 0.00, Val Loss: 5.89, Train BLEU: 0.00, Val BLEU: 3.04, Minutes Elapsed: 22.62
Sampling from val predictions...
Source: đây là một bản_vẽ cách để khảo_sát toàn xã_hội bởi_lẽ
Reference: this is a blueprint how to survey your society
Model: <SOS> it &apos;s a a a a , , ,
Attention Weights: tensor([[0.0541, 0.4531, 0.3497, 0.0727, 0.0493, 0.0179, 0.0026, 0.0004, 0.0001,
         0.0000],
        [0.0902, 0.5222, 0.3071, 0.0468, 0.0247, 0.0076, 0.0012, 0.0002, 0.0000,
         0.0000],
        [0.0204, 0.2503, 0.3222, 0.1230, 0.1560, 0.1010, 0.0218, 0.0040, 0.0007,
         0.0006],
        [0.0011, 0.0062, 0.0132, 0.0205, 0.0796, 0.2727, 0.2488, 0.1784, 0.0753,
         0.1044],
        [0.0001, 0.0003, 0.0009, 0.0024, 0.0124, 0.0709, 0.1311, 0.2167, 0.2106,
         0.3546],
        [0.0000, 0.0002, 0.0005, 0.0016, 0.0087, 0.0522, 0.1076, 0.2028, 0.2336,
         0.3926],
        [0.0000, 0.0002, 0.0005, 0.0016, 0.0083, 0.0486, 0.1017, 0.1977, 0.2387,
         0.

Epoch: 0.48, Train Loss: 0.00, Val Loss: 5.84, Train BLEU: 0.00, Val BLEU: 2.49, Minutes Elapsed: 25.09
Sampling from val predictions...
Source: đủ để trồng <UNK> triệu cây cà_chua . <EOS> <PAD>
Reference: that &apos;s enough space to plant <UNK> million tomato
Model: <SOS> we , , , , . . . <EOS>
Attention Weights: tensor([[0.5705, 0.3861, 0.0112, 0.0291, 0.0019, 0.0007, 0.0004, 0.0002, 0.0001,
         0.0000],
        [0.2897, 0.5732, 0.0453, 0.0887, 0.0027, 0.0004, 0.0001, 0.0000, 0.0000,
         0.0000],
        [0.0007, 0.0051, 0.0090, 0.8024, 0.1100, 0.0459, 0.0191, 0.0061, 0.0016,
         0.0000],
        [0.0000, 0.0002, 0.0004, 0.3368, 0.1461, 0.1915, 0.2078, 0.0990, 0.0182,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0076, 0.0144, 0.0694, 0.2765, 0.4591, 0.1730,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0007, 0.0021, 0.0141, 0.0950, 0.4165, 0.4716,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0003, 0.0010, 0.0070, 0.0528, 0.3265, 0.6124,
        

Epoch: 0.53, Train Loss: 0.00, Val Loss: 5.77, Train BLEU: 0.00, Val BLEU: 3.75, Minutes Elapsed: 27.59
Sampling from val predictions...
Source: peter <UNK> là một giáo_sư triết_học trước khi làm công_việc
Reference: peter <UNK> was a professor of philosophy before becoming
Model: <SOS> but we we we we the the , the
Attention Weights: tensor([[1.0446e-03, 8.9460e-01, 1.0348e-01, 8.3351e-04, 1.8015e-05, 2.4278e-05,
         5.5912e-06, 1.3471e-06, 1.1400e-06, 5.8245e-07],
        [1.6179e-03, 9.0266e-01, 9.5224e-02, 4.8784e-04, 4.5768e-06, 2.3822e-06,
         2.5679e-07, 3.2492e-08, 1.7214e-08, 6.7119e-09],
        [6.8701e-06, 8.1341e-01, 1.8623e-01, 3.4490e-04, 8.0103e-07, 3.8665e-07,
         2.8461e-08, 3.1339e-09, 1.7809e-09, 7.2869e-10],
        [5.3901e-07, 2.0118e-01, 7.7351e-01, 2.5129e-02, 1.1293e-04, 7.4057e-05,
         1.8909e-06, 5.5528e-08, 1.9241e-08, 4.2645e-09],
        [4.7231e-07, 1.0270e-02, 4.9072e-01, 3.3483e-01, 3.2930e-02, 1.2297e-01,
         7.9903e-03, 2.146

In [ ]:
experiment_results = load_experiment_log(experiment_name=EXPERIMENT_NAME)

In [ ]:
plot_single_learning_curve(experiment_results[0]['results'])

In [ ]:
summarize_results(experiment_results)[['best_val_loss', 'best_val_bleu', 'runtime', 
                                       'total_params', 'trainable_params', 'dt_created']]

In [ ]:
# reload model and test 
checkpoint = torch.load('model_checkpoints/{}.pth.tar'.format(MODEL_NAME), map_location=device)
model.load_state_dict(checkpoint)